In [ ]:
# default_exp storm
from nbdev import *

In [ ]:
#export
import numpy as np
import xarray as xr
from tqdm import tqdm
import logging

from xarray.core.variable import Variable

logger = logging.getLogger(__name__)
logging.basicConfig(format='%(levelname)s: %(message)s', level=logging.INFO)

try:
    import dask
except:
    logger.warning("Dask is not installed in your python environment. Xarray Dataset parallel computing will not work.")


# plotting
try:
    import matplotlib.pyplot as plt
    #import cartopy.crs as ccrs
except:
    logger.warning("Matplotlib and/or Cartopy is not installed in your python environment. Xarray Dataset plotting functions will not work.")

# Stormtracks

> calculate stormtracks from SLP fields

In [ ]:
#export
class storm():
    """The constructor for storm track class. Initialize a storm instance.
        
        If filename is given, try to load it directly.
        Arguments to the load function can be passed as key=value argument.
        Parameters
        ----------
            filename : string
                Datapath + filename.nc
            ds : dataset
                xarray dataset
            anomaly: bool
                calculates anomaly in time
    """
    def __init__(self, filename="", ds=None, anomaly = True, **kwargs):
        
        self._anomaly = anomaly
        
        if not filename:
            if ds is None:
                self.ds = None
            else:
                self.ds = ds
            return
        
        try:
            self.ds = None
            self.read(filename, **kwargs)
        except (OSError, IOError, RuntimeError):
            try:
                self.read(filename, **kwargs)
            except Exception:
                raise IOError("Unkown fileformat. Known formats are netcdf.")

    # Imported methods
    
    from drcstools.derivative import derivative 
    from drcstools.derivative import harmonicRegression
    from drcstools.derivative import calc_dX
      
    def __repr__(self):
        try:
            string = "\
            Xarray dataset with {} time steps. \n\
            Available fields: {}".format(
                self.ntime, ", ".join(self.variables)
            )
        except AttributeError:
            # Assume it's an empty Blocking()
            string = "\
            Empty contrack container.\n\
            Hint: use read() to load data."
        return string

    def __str__(self):
        return 'Class {}: \n{}'.format(self.__class__.__name__, self.ds)
  
    def __len__(self):
        return len(self.ds)
    
    def __getattr__(self, attr):
        if attr in self.__dict__:
            return getattr(self, attr)
        return getattr(self.ds, attr)

    
    def __getitem__(self, key):
        return self.ds[key]

    @property
    def ntime(self):
        """Return the number of time steps"""
        if len(self.ds.dims) != 3:
            logger.warning(
                "\nBe careful with the dimensions, "
                "you want dims = 3 and shape:\n"
                "(latitude, longitude, time)"
            )
            return self.ds.dims[self._get_name_time()]
        return self.ds.dims[self._get_name_time()]

    @property
    def variables(self):
        """Return the names of the variables"""
        return list(self.ds.data_vars)
    
    @property
    def dimensions(self):
        """Return the names of the dimensions"""
        return list(self.ds.dims)
    
    @property
    def grid(self):
        """Return the number of longitude and latitude grid"""
        if len(self.ds.dims) != 3:
            logger.warning(
                "\nBe careful with the dimensions, "
                "you want dims = 3 and shape:\n"
                "(latitude, longitude, time)"
            )
            return None
        string = "\
        latitude: {} \n\
        longitude: {}".format(
            self.ds.dims[self._get_name_latitude()], self.ds.dims[self._get_name_longitude()]
        ) 
        print(string)

    @property
    def dataset(self):
        """Return the dataset"""
        return self.ds

# ----------------------------------------------------------------------------
# Read / Import / Save data
    
    def read(self, filename, **kwargs):
        """
        Reads a file into a xarray dataset.
        
        Parameters
        ----------
            filename : string
                Valid path + filename
        """
        if self.ds is None:
            self.ds = xr.open_dataset(filename, **kwargs)
            if self._anomaly == True:
                self.ds = self.ds - self.ds.mean(self._get_name_time())
            logger.debug('read: {}'.format(self.__str__))
        else:
            errmsg = 'contrack() is already set!'
            raise ValueError(errmsg)
            
    def read_xarray(self, ds):
        """
        Read an existing xarray data set.
        
        Parameter:
        ----------
            ds: data set
                Valid xarray data set.
        """
        if self.ds is None:
            if not isinstance(ds, xr.core.dataset.Dataset):
                errmsg = 'ds has to be a xarray data set!'
                raise ValueError(errmsg)
            self.ds = ds
            logger.debug('read_xarray: {}'.format(self.__str__))
        else:
            errmsg = 'contrack() is already set!'
            raise ValueError(errmsg)
 
# Set up / Check dimensions
   
    def set_up(self,
               time_name=None,
               longitude_name=None,
               latitude_name=None,
               force=False,
               write=True
    ):
        """
        Prepares the dataset for storm tracking. Does consistency checks
        and tests if all required information is available. Sets (automatically 
        or manually) internal variables and dimensions.
        Parameters
        ----------
            time_name : string, optional
                Name of time dimension. The default is None.
            longitude_name : string, optional
                Name of longitude dimension. The default is None.
            latitude_name : string, optional
                Name of latitude dimension. The default is None.
            force=False: bool, optional 
                Skip some consistency checks.
            write=True: bool, optional
                Print name of dimensions.
        Returns
        -------
            None.
        """

        # set dimensions
        if time_name is None:
            self._time_name = self._get_name_time()  
        else:
            self._time_name = time_name
        if longitude_name is None:
            self._longitude_name = self._get_name_longitude()
        else:
            self._longitude_name = longitude_name
        if latitude_name is None:
            self._latitude_name = self._get_name_latitude()
        else:
            self._latitude_name = latitude_name

        # set resolution
        if (self._longitude_name and self._latitude_name) is not None:
            self._dlon =  self._get_resolution(self._longitude_name, force=force)
            self._dlat =  self._get_resolution(self._latitude_name, force=force)

        if self._time_name is not None:
            self._dtime = self._get_resolution(self._time_name, force=force)
       
        # Transpose data
        self.ds = self.ds.transpose(self._time_name,
                                    self._latitude_name,
                                    self._longitude_name) 
        # print names    
        if write:
            logger.info(
                "\n time: '{}'\n"
                " longitude: '{}'\n"
                " latitude: '{}'\n".format(
                self._time_name, 
                self._longitude_name,
                self._latitude_name)
            )

    
    def _get_name_time(self):
        """
        check for 'time' dimension and return name
        """
        # check unit
        for dim in self.ds.dims:
            if (('units' in self.ds[dim].attrs and
                'since' in self.ds[dim].attrs['units']) or 
                ('units' in self.ds[dim].encoding and
                 'since' in self.ds[dim].encoding['units']) or
                dim in ['time']):
                return dim
        # check dtype
        for dim in self.ds.variables:
            try:
                var = self.ds[dim].data[0]
            except IndexError:
                var = self.ds[dim].data
            if isinstance(var, datetime64):
                return dim   
        # no 'time' dimension found
        logger.warning(
            "\n 'time' dimension (dtype='datetime64[ns]') not found."
        )
        return None     


    def _get_name_longitude(self):
        """
        check for 'longitude' dimension and return name
        """
        for dim in self.ds.dims:
            if (('units' in self.ds[dim].attrs and
               self.ds[dim].attrs['units'] in ['degree_east', 'degrees_east']) or
               dim in ['lon', 'longitude', 'x']):
               return dim
        # no 'longitude' dimension found
        logger.warning(
            "\n 'longitude' dimension (unit='degrees_east') not found."
        )
        return None


    def _get_name_latitude(self):
        """
        check for 'latitude' dimension and return name
        """
        for dim in self.ds.dims:
            if (('units' in self.ds[dim].attrs  and
                self.ds[dim].attrs['units'] in ['degree_north', 'degrees_north']) or
                dim in ['lat', 'latitude', 'y']):
                return dim
        # no 'latitude' dimension found
        logger.warning(
            "\n 'latitude' dimension (unit='degrees_north') not found."
        )
        return None
            
    def _get_resolution(self, dim, force=False):
        """
        set spatial (lat/lon) and temporal (time) resolution
        """
        # time dimension in hours
        if dim == self._time_name:
            try:
                var = self.ds[dim].to_index()
                delta = np.unique((
                    self.ds[dim].to_index()[1:] - 
                    self.ds[dim].to_index()[:-1])
                    .astype('timedelta64[h]')
                )
            except AttributeError:  # dates outside of normal range
                # we can still move on if the unit is "days since ..."
                if ('units' in self.ds[dim].attrs and
                    'days' in self.ds[dim].attrs['units']):
                    var = self.ds[dim].data
                    delta = np.unique(var[1:] - var[:-1])
                else:
                    errmsg = 'Can not decode time with unit {}'.format(
                        self.ds[dim].attrs['units'])
                    raise ValueError(errmsg)
        # lat/lon dimension in Degree
        else:
            delta = abs(np.unique((
                self.ds[dim].data[1:] - 
                self.ds[dim].data[:-1])
            ))
        # check resolution
        if len(delta) > 1:
            errmsg = 'No regular grid found for dimension {}.\n\
            Hint: use set_up(force=True).'.format(dim)
            if force and dim != self._time_name:
                logging.warning(errmsg)
                logmsg = ' '.join(['force=True: using mean of non-equidistant',
                                   'grid {}'.format(delta)])
                logging.warning(logmsg)
                delta = round(delta.mean(), 2)
            else:
                if dim == self._time_name:
                    logging.warning(errmsg)
                else:
                    raise ValueError(errmsg)
        elif delta[0] == 0:
            errmsg = 'Two equivalent values found for dimension {}.'.format(
                dim)
            raise ValueError(errmsg)
        elif delta[0] < 0:
            errmsg = ' '.join(['{} not increasing. This should',
                                   'not happen?!']).format(dim)
            raise ValueError(errmsg)
            
        return delta

ImportError: cannot import name 'derivative' from 'drcstools.derivative' (/mnt/c/Users/flori/Documents/git_repositories/drcstools/drcstools/derivative.py)

In [ ]:
show_doc(storm.read)

<h4 id="storm.read" class="doc_header"><code>storm.read</code><a href="__main__.py#L119" class="source_link" style="float:right">[source]</a></h4>

> <code>storm.read</code>(**`filename`**, **\*\*`kwargs`**)

Reads a file into a xarray dataset.

Parameters
----------
    filename : string
        Valid path + filename

In [ ]:
show_doc(storm.read_xarray)

<h4 id="storm.read_xarray" class="doc_header"><code>storm.read_xarray</code><a href="__main__.py#L137" class="source_link" style="float:right">[source]</a></h4>

> <code>storm.read_xarray</code>(**`ds`**)

Read an existing xarray data set.

Parameter:
----------
    ds: data set
        Valid xarray data set.

In [ ]:
show_doc(storm.set_up)

<h4 id="storm.set_up" class="doc_header"><code>storm.set_up</code><a href="__main__.py#L158" class="source_link" style="float:right">[source]</a></h4>

> <code>storm.set_up</code>(**`time_name`**=*`None`*, **`longitude_name`**=*`None`*, **`latitude_name`**=*`None`*, **`force`**=*`False`*, **`write`**=*`True`*)

Prepares the dataset for storm tracking. Does consistency checks
and tests if all required information is available. Sets (automatically 
or manually) internal variables and dimensions.
Parameters
----------
    time_name : string, optional
        Name of time dimension. The default is None.
    longitude_name : string, optional
        Name of longitude dimension. The default is None.
    latitude_name : string, optional
        Name of latitude dimension. The default is None.
    force=False: bool, optional 
        Skip some consistency checks.
    write=True: bool, optional
        Print name of dimensions.
Returns
-------
    None.

In [ ]:
show_doc(storm._get_resolution)

<h4 id="storm._get_resolution" class="doc_header"><code>storm._get_resolution</code><a href="__main__.py#L282" class="source_link" style="float:right">[source]</a></h4>

> <code>storm._get_resolution</code>(**`dim`**, **`force`**=*`False`*)

set spatial (lat/lon) and temporal (time) resolution

In [ ]:
show_doc(storm._get_name_time)

<h4 id="storm._get_name_time" class="doc_header"><code>storm._get_name_time</code><a href="__main__.py#L224" class="source_link" style="float:right">[source]</a></h4>

> <code>storm._get_name_time</code>()

check for 'time' dimension and return name

In [ ]:
show_doc(storm._get_name_longitude)

<h4 id="storm._get_name_longitude" class="doc_header"><code>storm._get_name_longitude</code><a href="__main__.py#L251" class="source_link" style="float:right">[source]</a></h4>

> <code>storm._get_name_longitude</code>()

check for 'longitude' dimension and return name

In [ ]:
show_doc(storm._get_name_longitude)

<h4 id="storm._get_name_longitude" class="doc_header"><code>storm._get_name_longitude</code><a href="__main__.py#L251" class="source_link" style="float:right">[source]</a></h4>

> <code>storm._get_name_longitude</code>()

check for 'longitude' dimension and return name